<a href="https://colab.research.google.com/github/ZerocoderUniversity/AI-course-prompt-engineering/blob/main/07Project_video_processing_ai/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотеки

In [ ]:
!pip install requests
!pip install moviepy

In [ ]:
# Импорт модулей
import requests
import os
from moviepy.editor import VideoFileClip, CompositeVideoClip, ImageClip

# Код

In [ ]:
# Настройки
PIXABAY_API_KEY = '47304130-4b574d966d6ab0671cc18abfa'  # Замените на ваш API ключ

search_query = 'Data Scientist'  # Ваш поисковый запрос
per_page = 3  # Количество видео для загрузки

logo_path = '/content/google.png'  # Путь к вашему логотипу

# Функции
def download_pixabay_videos(query, per_page=3):
    url = 'https://pixabay.com/api/videos/'

    params = {
        'key': PIXABAY_API_KEY,
        'q': query,
        'per_page': per_page,
        'safesearch': 'true'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        videos = response.json()['hits']
        video_paths = []
        for idx, video in enumerate(videos):
            # Выбираем видео с наилучшим качеством
            video_files = video['videos']
            if 'large' in video_files:
                video_url = video_files['large']['url']
            elif 'medium' in video_files:
                video_url = video_files['medium']['url']
            else:
                video_url = video_files['small']['url']

            video_response = requests.get(video_url)
            video_filename = f'video_{idx}.mp4'
            with open(video_filename, 'wb') as f:
                f.write(video_response.content)
            video_paths.append(video_filename)
        return video_paths
    else:
        print('Ошибка при запросе к Pixabay API:', response.status_code)
        return []

def overlay_logo_on_video(video_path, logo_path, output_path):
    # Загрузка видео
    video_clip = VideoFileClip(video_path)

    # Проверка наличия аудио (для избежания предупреждений)
    if video_clip.audio is None:
        video_clip = video_clip.set_audio(None)

    # Загрузка логотипа
    logo = (ImageClip(logo_path)
            .set_duration(video_clip.duration)
            .resize(height=100)  # Измените размер логотипа при необходимости
            .margin(right=8, top=8, opacity=0)  # Отступы
            .set_pos(("right", "top")))  # Позиция логотипа

    # Наложение логотипа на видео
    final_video = CompositeVideoClip([video_clip, logo])

    # Сохранение итогового видео
    final_video.write_videofile(output_path, codec='libx264', audio_codec='aac')

    # Освобождение ресурсов
    video_clip.close()
    final_video.close()

# Основной процесс

video_paths = download_pixabay_videos(search_query, per_page)

if video_paths:
    for video_path in video_paths:
        output_path = f'logo_{video_path}'
        overlay_logo_on_video(video_path, logo_path, output_path)
        print(f'Видео с наложенным логотипом сохранено: {output_path}')
else:
    print('Не удалось скачать видео.')

# Совмещение

In [ ]:
# Импорт модулей
from moviepy.editor import VideoFileClip, clips_array

# Настройки
video_paths = ['/content/logo_video_0.mp4', '/content/logo_video_1.mp4', '/content/logo_video_2.mp4']  # Замените на ваши файлы
output_path = 'merged_video.mp4'

# Загружаем видеофайлы
video_clips = [VideoFileClip(video) for video in video_paths]

target_size = (640, 360)  # Задайте нужный размер
resized_clips = [clip.resize(target_size) for clip in video_clips]

# Объединяем видео в один кадр (в ряд)
final_clip = clips_array([resized_clips])

# Сохраняем итоговое видео
final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

# Освобождаем ресурсы
for clip in resized_clips:
    clip.close()
final_clip.close()

print(f'Итоговое видео сохранено как {output_path}')